In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
## libraries
import pandas as pd

In [3]:
## data import
path_to_treat = PATH_TO_TREAT ## for import
path_treated = PATH_TREATED ## for export
product = pd.read_csv(path_to_treat+'mapping_product.csv', delimiter = ';',encoding = "ISO-8859-1")
support = pd.read_csv(path_to_treat+'mapping_support.csv', delimiter = ';',encoding = "ISO-8859-1")
customer = pd.read_csv(path_to_treat+'table_customer_profile.csv', delimiter = ';', encoding = "UTF-8", dtype=str,decimal=',') ## dtype forced to string because NACE_CODE and POSTCODE have uncommon values
transactions = pd.read_csv(path_to_treat+'table_transactions.csv', delimiter = ';',encoding = "ISO-8859-1",decimal=',')

# Analysis of mapping_product

In [4]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8472 entries, 0 to 8471
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   PRODUCT_CODE               8472 non-null   object
 1   PRODUCT_GRAND_FAMILY_CODE  8294 non-null   object
 2   PRODUCT_GRAND_FAMILY_NAME  8294 non-null   object
 3   PRODUCT_FAMILY_CODE        8294 non-null   object
 4   PRODUCT_FAMILY_NAME        8294 non-null   object
dtypes: object(5)
memory usage: 331.1+ KB


In [5]:
product.head(5)

,PRODUCT_CODE,PRODUCT_GRAND_FAMILY_CODE,PRODUCT_GRAND_FAMILY_NAME,PRODUCT_FAMILY_CODE,PRODUCT_FAMILY_NAME
0,732,GF05,Frais de services,FA03,Abonnements
1,733,GF04,Produits Hors Station,FA05,Tr Autres tiers - Ristournes et commissions
2,734,GF04,Produits Hors Station,FA05,Tr Autres tiers - Ristournes et commissions
3,735,GF07,Caution et Garantie,FA15,CA Tr Autres tiers - Caution/Garantie
4,738,GF04,Produits Hors Station,FA17,Tr Parkings


In [6]:
print(product.shape) ## number of rows and columns
product.isna().sum()

(8472, 5)


PRODUCT_CODE                   0
PRODUCT_GRAND_FAMILY_CODE    178
PRODUCT_GRAND_FAMILY_NAME    178
PRODUCT_FAMILY_CODE          178
PRODUCT_FAMILY_NAME          178
dtype: int64

In [7]:
product[product.isna().any(axis=1)]
## 178 rows contain only NaN values, we should regroup them

,PRODUCT_CODE,PRODUCT_GRAND_FAMILY_CODE,PRODUCT_GRAND_FAMILY_NAME,PRODUCT_FAMILY_CODE,PRODUCT_FAMILY_NAME
288,J350,NaN,NaN,NaN,NaN
294,M000,NaN,NaN,NaN,NaN
295,M010,NaN,NaN,NaN,NaN
297,M200,NaN,NaN,NaN,NaN
298,M300,NaN,NaN,NaN,NaN
...,...,...,...,...,...
8453,ADC1,NaN,NaN,NaN,NaN
8454,ADC7,NaN,NaN,NaN,NaN
8458,ADC2,NaN,NaN,NaN,NaN
8459,8987,NaN,NaN,NaN,NaN


In [8]:
product['PRODUCT_GRAND_FAMILY_CODE'].value_counts(dropna = False)

GF05    7816
GF04     214
NaN      178
GF01     103
GF03      62
GF02      36
GFZZ      31
GF06      20
GF07      12
Name: PRODUCT_GRAND_FAMILY_CODE, dtype: int64

In [9]:
product['PRODUCT_GRAND_FAMILY_NAME'].value_counts(dropna = False)

Frais de services                       7816
Produits Hors Station                    214
NaN                                      178
Produits stations carburants             103
Produits Station Boutique                 62
Produits Station Lavage                   36
Grande Famille Produits Non Utilisés      31
Frais en clientèle                        20
Caution et Garantie                       12
Name: PRODUCT_GRAND_FAMILY_NAME, dtype: int64

In [10]:
product['PRODUCT_FAMILY_CODE'].value_counts(dropna = False)

FA03    7657
NaN      178
FA16     120
FA22     112
FA01      87
FA02      62
FA26      34
FAZZ      31
FA04      30
FA08      30
FA18      29
FA06      18
FA17      17
FA25      16
FA15      12
FA21      10
FA10       9
FA05       5
FA19       4
FA09       3
FA23       2
FA27       2
FA36       1
FA11       1
FA24       1
FA07       1
Name: PRODUCT_FAMILY_CODE, dtype: int64

In [11]:
product['PRODUCT_FAMILY_NAME'].value_counts(dropna = False)

Abonnements                                     7657
NaN                                              178
Tr Péages                                        120
Frais à l acte                                   112
Transactions stations carburants                  87
Transactions Station Boutique                     62
Transactions Station Lavage                       34
Famille Produits Non Utilisés                     31
Tr Autres tiers - Services Assistance 24/24       30
Frais sur transactions Péages                     30
Tr Autres tiers - CA C.H.R.                       29
Remboursement divers                              18
Tr Parkings                                       17
Ristournes et commissions carburants              16
CA Tr Autres tiers - Caution/Garantie             12
Frais sur transactions Stations                   10
Tr Electriques                                     9
Tr Autres tiers - Ristournes et commissions        5
Ecotaxe                                       

In [12]:
## 1) delete PRODUCT_GRAND_FAMILY_NAME and PRODUCT_FAMILY_NAME
product2 = product.drop(columns=['PRODUCT_GRAND_FAMILY_NAME','PRODUCT_FAMILY_NAME'])
## 2) change NaN to GFNA and FANA
product2 = product2.fillna(value={'PRODUCT_GRAND_FAMILY_CODE': 'GFNA', 'PRODUCT_FAMILY_CODE': 'FANA'})
## 3) sort by PRODUCT_CODE for clarity
product2 = product2.sort_values(['PRODUCT_GRAND_FAMILY_CODE','PRODUCT_FAMILY_CODE','PRODUCT_CODE'])

In [13]:
product2.to_csv(path_treated+'mapping_product2.csv', sep =";", index=False)

In [14]:
product2.head(5)

,PRODUCT_CODE,PRODUCT_GRAND_FAMILY_CODE,PRODUCT_FAMILY_CODE
86,"1,00E+02",GF01,FA01
90,101,GF01,FA01
91,102,GF01,FA01
92,103,GF01,FA01
93,104,GF01,FA01


In [15]:
product2['PRODUCT_CODE'].value_counts(dropna = False).shape[0] == product2.shape[0]
## all PRODUCT_CODE values are unique

True

# Analysis of mapping_support

In [16]:
support.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112145 entries, 0 to 2112144
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   EP_CODE            int64 
 1   SUPPORT_CODE       object
 2   SUPPORT_TYPE       object
 3   SUPPORT_STATUS     object
 4   SUPPORT_SUBSTATUS  object
 5   VEHICLE_TYPE       object
dtypes: int64(1), object(5)
memory usage: 96.7+ MB


In [17]:
print(support.shape)
support.head(5)

(2112145, 6)


,EP_CODE,SUPPORT_CODE,SUPPORT_TYPE,SUPPORT_STATUS,SUPPORT_SUBSTATUS,VEHICLE_TYPE
0,87701981,C700,C,Actif,En activité,03-PASSENGER VEHICLE
1,87701929,C700,C,Actif,En activité,04-OTHER
2,87701879,C700,C,Actif,En activité,03-PASSENGER VEHICLE
3,87701871,C700,C,Actif,En activité,03-PASSENGER VEHICLE
4,87701624,C700,C,Actif,En activité,03-PASSENGER VEHICLE


In [18]:
## EP_CODE : problem, many repetitions, with sometimes different vehicle types
support.loc[(support['EP_CODE'] == 87701925)].value_counts()
## we should delete duplicates as it doesn't give more informations

EP_CODE   SUPPORT_CODE  SUPPORT_TYPE  SUPPORT_STATUS  SUPPORT_SUBSTATUS  VEHICLE_TYPE                          
87701925  C100          C             Actif           En activité        03-PASSENGER VEHICLE                      114
                                                                         04-OTHER                                    4
                                                                         01-HEAVY GOODS VEHICLE                      2
                                                                         02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE      1
dtype: int64

In [19]:
support['SUPPORT_CODE'].value_counts(dropna=False)

C100    838089
C700    750228
TIS4    166175
C200    123913
TIS7     54225
T200     48298
PL01     29636
R200     29224
PL03     21253
F200      9313
H200      8668
CA00      7997
C800      5731
X600      5393
X700      4371
C901      1251
PL31      1118
WK01      1056
XF00       989
CB00       899
C500       822
WK11       802
WK00       687
X800       524
X900       316
CC00       314
G100       271
G200       104
CE00        95
WK10        76
GE00        74
G400        66
C900        43
H700        39
PL41        37
XD00        25
G300        13
H100         5
X500         3
TIS5         1
NaN          1
Name: SUPPORT_CODE, dtype: int64

In [20]:
support['SUPPORT_TYPE'].value_counts(dropna=False)
## Three categories => we should one-hot encode it??

C    1837078
V     223023
P      52044
Name: SUPPORT_TYPE, dtype: int64

In [21]:
support['SUPPORT_STATUS'].value_counts()
### Only "Actif" => we should remove it

Actif    2112145
Name: SUPPORT_STATUS, dtype: int64

In [22]:
support['SUPPORT_SUBSTATUS'].value_counts()
## Only "En activité" => we should remove it

En activité    2112145
Name: SUPPORT_SUBSTATUS, dtype: int64

In [23]:
support['VEHICLE_TYPE'].value_counts(dropna=False)
## 4 categories => we should one-hot encode it??

02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE    725593
03-PASSENGER VEHICLE                      639872
04-OTHER                                  596072
01-HEAVY GOODS VEHICLE                    150606
NaN                                            2
Name: VEHICLE_TYPE, dtype: int64

In [24]:
## there are some NA values to treat
print(support.shape[0]) ## number of rows
support.isna().sum()

2112145


EP_CODE              0
SUPPORT_CODE         1
SUPPORT_TYPE         0
SUPPORT_STATUS       0
SUPPORT_SUBSTATUS    0
VEHICLE_TYPE         2
dtype: int64

In [25]:
support[support.isna().any(axis=1)]
## 2 observations contain NA :
## don't remove them because of inconsistency with EP_CODE from transaction table

,EP_CODE,SUPPORT_CODE,SUPPORT_TYPE,SUPPORT_STATUS,SUPPORT_SUBSTATUS,VEHICLE_TYPE
892388,87211624,C700,C,Actif,En activité,NaN
1468215,87361287,NaN,V,Actif,En activité,NaN


In [26]:
## analysis of NA values : EP_CODE 87211624
support.loc[(support['EP_CODE'] == 87211624)].value_counts()
## VEHICLE_TYPE can be replaced by 04-OTHER

EP_CODE   SUPPORT_CODE  SUPPORT_TYPE  SUPPORT_STATUS  SUPPORT_SUBSTATUS  VEHICLE_TYPE                          
87211624  C700          C             Actif           En activité        02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE    161
                                                                         04-OTHER                                   38
                                                                         01-HEAVY GOODS VEHICLE                     11
                                                                         03-PASSENGER VEHICLE                        8
          TIS4          V             Actif           En activité        03-PASSENGER VEHICLE                        1
dtype: int64

In [27]:
## analysis of NA values : EP_CODE 87361287
support.loc[(support['EP_CODE'] == 87361287)]
## VEHICLE_TYPE can be replaced by 04-OTHER
## SUPPORT_CODE can be replaced by TIS4 (most frequent SUPPORT_CODE for SUPPORT_TYPE V)

,EP_CODE,SUPPORT_CODE,SUPPORT_TYPE,SUPPORT_STATUS,SUPPORT_SUBSTATUS,VEHICLE_TYPE
1468215,87361287,NaN,V,Actif,En activité,NaN
1566464,87361287,X700,C,Actif,En activité,03-PASSENGER VEHICLE


In [28]:
## 1) delete SUPPORT_STATUS and SUPPORT_SUBSTATUS as they don't give different values
support2 = support.drop(columns=['SUPPORT_STATUS','SUPPORT_SUBSTATUS'])
## 2) treat NAs as explained earlier
support2 = support2.fillna(value={'SUPPORT_CODE': 'TIS4', 'VEHICLE_TYPE': '04-OTHER'})
## 3) treat repetitions of EP_CODE
support2 = support2.drop_duplicates()
## 4) order by EP_CODE for visual clarity, then by VEHICLE_TYPE
support2 = support2.sort_values(by=['EP_CODE','VEHICLE_TYPE'])

In [29]:
support2.to_csv(path_treated+'mapping_support2.csv', sep =";", index=False)

In [30]:
support2.head(5)

,EP_CODE,SUPPORT_CODE,SUPPORT_TYPE,VEHICLE_TYPE
199217,87000052,C700,C,01-HEAVY GOODS VEHICLE
199216,87000052,C700,C,02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE
1014547,87000070,C700,C,01-HEAVY GOODS VEHICLE
888028,87000070,TIS4,V,02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE
1014549,87000070,C700,C,02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE


In [31]:
print(support2['EP_CODE'].value_counts(dropna = False).shape[0])
print(support2.shape[0])
## not the same shape, because some EP_CODE have several VEHICLE_TYPE

149384
288969


In [32]:
support2.loc[(support2['EP_CODE'] == 87701925)].value_counts()
## from the previous example, we know have only 1 row for each VEHICLE_TYPE

EP_CODE   SUPPORT_CODE  SUPPORT_TYPE  VEHICLE_TYPE                          
87701925  C100          C             04-OTHER                                  1
                                      03-PASSENGER VEHICLE                      1
                                      02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE    1
                                      01-HEAVY GOODS VEHICLE                    1
dtype: int64

# Analysis of table_customers

In [33]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144499 entries, 0 to 144498
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   EP_CODE             144499 non-null  object
 1   EP_STATUS           144499 non-null  object
 2   EP_TYPE             144490 non-null  object
 3   EP_NACE_CODE        144499 non-null  object
 4   EP_NACE_NAME        144499 non-null  object
 5   EP_DATE_ACTIVATION  144499 non-null  object
 6   EP_DATE_END         144477 non-null  object
 7   EP_DATE_LAST_INV    144499 non-null  object
 8   EP_DATE_LAST_TR     143820 non-null  object
 9   EP_POSTCODE         144499 non-null  object
 10  EP_CITY             144499 non-null  object
 11  EP_COUNTRY          144499 non-null  object
 12  EP_SALES_TEAM       144499 non-null  object
 13  EP_FLEET_TYPE       144499 non-null  object
dtypes: object(14)
memory usage: 15.4+ MB


In [34]:
print(customer.shape)
customer.head(5)

(144499, 14)


,EP_CODE,EP_STATUS,EP_TYPE,EP_NACE_CODE,EP_NACE_NAME,EP_DATE_ACTIVATION,EP_DATE_END,EP_DATE_LAST_INV,EP_DATE_LAST_TR,EP_POSTCODE,EP_CITY,EP_COUNTRY,EP_SALES_TEAM,EP_FLEET_TYPE
0,87261819,Activé,Standard,6420,Activités des sociétés holding,19/09/2016,30/09/2024,31/05/2021,12/06/2021,44308,NANTES CEDEX 3,FRANCE,Ventes Nationales : Région Ouest,VL_MIXTE
1,87189132,Activé,Standard,8690,Autres activité pour la santé humaine,23/05/2013,31/05/2025,31/05/2021,12/06/2021,13100,LE THOLONET,FRANCE,Ventes Nationales : Régions Sud,VL_PUR
2,87642229,Activé,Administration,6820,Location et exploitation de biens immobiliers ...,24/02/1995,28/02/2023,31/05/2021,11/06/2021,54054,NANCY CEDEX,FRANCE,Ventes Nationales : Marchés Publics,VL_PUR
3,87036107,Activé,Standard,3320,Installation de machines et d'équipements indu...,19/01/2001,31/01/2024,31/05/2021,11/06/2021,27930,GUICHAINVILLE,FRANCE,Ventes Locales : Région Nord,VL_MIXTE
4,87176302,Activé,Standard,7732,Location et location bail de machines et équip...,05/02/2003,28/02/2023,31/05/2021,11/06/2021,31022,TOULOUSE,FRANCE,Ventes Nationales : Région Nord-Est,VL_MIXTE


In [35]:
print(customer.shape[0]) ## number of rows
customer.isna().sum()

144499


EP_CODE                 0
EP_STATUS               0
EP_TYPE                 9
EP_NACE_CODE            0
EP_NACE_NAME            0
EP_DATE_ACTIVATION      0
EP_DATE_END            22
EP_DATE_LAST_INV        0
EP_DATE_LAST_TR       679
EP_POSTCODE             0
EP_CITY                 0
EP_COUNTRY              0
EP_SALES_TEAM           0
EP_FLEET_TYPE           0
dtype: int64

In [36]:
customer['EP_STATUS'].value_counts(dropna = False)

Activé                      143751
Opposé                         662
Suspendu                        82
*** Ne plus utiliser ***         4
Name: EP_STATUS, dtype: int64

In [37]:
customer['EP_TYPE'].value_counts(dropna = False)

Standard                      133115
Administration                 10225
International organisation       678
Third party intermediary         472
NaN                                9
Name: EP_TYPE, dtype: int64

In [38]:
customer.loc[customer['EP_TYPE'].isna()]

,EP_CODE,EP_STATUS,EP_TYPE,EP_NACE_CODE,EP_NACE_NAME,EP_DATE_ACTIVATION,EP_DATE_END,EP_DATE_LAST_INV,EP_DATE_LAST_TR,EP_POSTCODE,EP_CITY,EP_COUNTRY,EP_SALES_TEAM,EP_FLEET_TYPE
27,87210501,Activé,NaN,4941,Transports routiers de fret,14/04/2014,30/04/2022,31/05/2021,11/06/2021,94310,ORLY,FRANCE,Ventes Nationales : Région Ouest,VL_MIXTE
11922,87435177,Activé,NaN,8622,Activités médecins spécialistes,02/09/2016,30/06/2023,31/05/2021,09/06/2021,63000,CLERMONT FERRAND,FRANCE,Ventes Nationales : Grands Comptes,VL_PUR
55772,87233628,Activé,NaN,6202,Conseil informatique,27/02/2015,28/02/2023,30/04/2021,29/04/2021,14790,MOUEN,FRANCE,Ventes Locales : Région Ouest,VL_PUR
71281,87258629,Activé,NaN,8121,Nettoyage courant des bâtiments,30/06/2016,30/06/2024,31/05/2021,11/06/2021,62730,MARCK,FRANCE,Ventes Locales : Région Nord,VL_PUR
72640,87257669,Activé,NaN,4941,Transports routiers de fret,13/06/2016,30/06/2024,31/05/2021,12/06/2021,68100,MULHOUSE,FRANCE,Ventes Locales : Région Est,VL_MIXTE
78252,87230788,Activé,NaN,4332,Travaux de menuiserie,05/01/2015,31/01/2023,31/05/2021,11/06/2021,34800,LIEURAN CABRIERES,FRANCE,Ventes Locales : Région Sud,VL_MIXTE
101050,87246828,Activé,NaN,4120,Construction de bâtiments résidentiels et non ...,08/12/2015,31/12/2023,31/05/2021,12/06/2021,51370,ST BRICE COURCELLES,FRANCE,Ventes Locales : Région Est,VL_MIXTE
107748,87099981,Activé,NaN,2894,Fabrication de machines pour les industries te...,19/03/2007,31/03/2024,31/05/2021,10/06/2021,42390,VILLARS,FRANCE,Ventes Locales : Région Sud,VL_PUR
114105,87212402,Activé,NaN,5590,Autres hébergements,05/05/2014,31/05/2022,31/05/2021,18/05/2021,75012,PARIS,FRANCE,Ventes Locales : Région Est,VL_PUR


In [39]:
customer['EP_NACE_NAME'].value_counts(dropna = False)

Transports routiers de fret                                                      5525
Autres travaux de construction spécialisée n.c.a.                                5384
Installation électrique                                                          4931
Travaux de plomberie et installation de chauffage et de conditionnement d'air    4458
Administration publique générale                                                 3910
                                                                                 ... 
Fabrication de vêtements en cuir                                                    1
Fabrication de ficelles, cordes et filets                                           1
Restauration d'articles d'horlogerie et de bijouterie                               1
Caisses de retraite                                                                 1
Frappe de monnaie                                                                   1
Name: EP_NACE_NAME, Length: 586, dtype: int64

In [40]:
customer['EP_FLEET_TYPE'].value_counts(dropna = False)

VL_PUR      96536
VL_MIXTE    39836
PL_MIXTE     3605
PL           2672
PL_VIP       1850
Name: EP_FLEET_TYPE, dtype: int64

In [41]:
print(customer['EP_CODE'].value_counts(dropna = False).shape[0])
print(customer.shape[0])
## EP_CODE is a PRIMARY KEY for the table customer

144499
144499


In [42]:
## 1) delete all columns except EP_CODE, EP_TYPE and EP_FLEET_TYPE
customer2 = customer[['EP_CODE','EP_STATUS','EP_TYPE', 'EP_FLEET_TYPE']]
## 2) sort by EP_CODE
customer2 = customer2.sort_values('EP_CODE')

In [43]:
customer2.to_csv(path_treated+'table_customer_profile2.csv', sep =";", index=False, encoding='ISO-8859-1')

In [44]:
customer2.head(5)

,EP_CODE,EP_STATUS,EP_TYPE,EP_FLEET_TYPE
65449,87000019,Activé,Standard,VL_PUR
635,87000052,Activé,Standard,VL_MIXTE
27611,87000070,Activé,Standard,VL_MIXTE
97573,87000072,Activé,Administration,VL_MIXTE
59952,87000076,Activé,Standard,VL_PUR


# Analysis of table_transactions

##   Fuel transactions




In [45]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146085 entries, 0 to 1146084
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   EP_CODE          1146085 non-null  int64  
 1   TR_PRODUCT_CODE  1146085 non-null  int64  
 2   TR_PRODUCT_TYPE  1146085 non-null  object 
 3   TR_SUPPORT_CODE  1144477 non-null  object 
 4   TR_SUPPORT_TYPE  1143691 non-null  object 
 5   TR_DATE          1146085 non-null  object 
 6   TR_AMOUNT        1146085 non-null  float64
 7   TR_VOLUME        1146085 non-null  float64
 8   TR_COUNT         1146085 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 78.7+ MB


In [46]:
print(transactions.shape)
transactions.head(5)
## date should only be the day

(1146085, 9)


,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
0,87256967,730,Other,C100,C,"08/06/21 00:00:00,000000000",11.83,0.00,5
1,87147335,102,Fuel,C100,C,"09/06/21 00:00:00,000000000",791.47,628.76,11
2,87128463,731,Other,TIS4,V,"08/06/21 00:00:00,000000000",43.34,0.00,8
3,87235141,102,Fuel,C100,C,"09/06/21 00:00:00,000000000",1479.09,1172.73,25
4,87243619,730,Other,C100,C,"08/06/21 00:00:00,000000000",73.34,0.00,10


In [47]:
print(transactions.shape[0])
transactions.isna().sum()
## only TR_SUPPORT_CODE and TR_SUPPORT_TYPE have NA values

1146085


EP_CODE               0
TR_PRODUCT_CODE       0
TR_PRODUCT_TYPE       0
TR_SUPPORT_CODE    1608
TR_SUPPORT_TYPE    2394
TR_DATE               0
TR_AMOUNT             0
TR_VOLUME             0
TR_COUNT              0
dtype: int64

In [48]:
transactions[transactions.isna().any(axis=1)]

,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
795,87156258,888,Other,NaN,P,"07/06/21 00:00:00,000000000",4.00,0.00,1
1354,87989971,112,Fuel,EMET,NaN,"05/06/21 00:00:00,000000000",807.90,623.45,12
1416,87045820,878,Other,NaN,P,"07/06/21 00:00:00,000000000",60.00,0.00,1
2019,87361397,9060,Other,PL01,NaN,"07/06/21 00:00:00,000000000",3.00,0.00,1
2838,87773507,888,Other,NaN,P,"07/06/21 00:00:00,000000000",4.00,0.00,1
...,...,...,...,...,...,...,...,...,...
1141729,87183563,9076,Other,T200,NaN,"12/06/21 00:00:00,000000000",7.50,0.00,1
1142537,87980100,102,Fuel,EMET,NaN,"06/06/21 00:00:00,000000000",1353.23,1076.37,12
1142870,87216832,9076,Other,C100,NaN,"14/06/21 00:00:00,000000000",7.50,0.00,1
1145300,87989974,112,Fuel,EMET,NaN,"05/06/21 00:00:00,000000000",103.43,79.57,2


In [49]:
transactions['TR_PRODUCT_TYPE'].value_counts(dropna = False)
## we should replace it by a boolean

Fuel     729415
Other    416670
Name: TR_PRODUCT_TYPE, dtype: int64

In [50]:
transactions.loc[(transactions['TR_SUPPORT_CODE'].isna()) & (transactions['TR_SUPPORT_TYPE'].isna())]

,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
7106,87086526,878,Other,NaN,NaN,"07/06/21 00:00:00,000000000",24.0,0.0,1
14742,87274895,9010,Other,NaN,NaN,"07/06/21 00:00:00,000000000",0.1,0.0,1
20996,87036303,878,Other,NaN,NaN,"07/06/21 00:00:00,000000000",32.0,0.0,1
24875,87180347,888,Other,NaN,NaN,"07/06/21 00:00:00,000000000",40.0,0.0,1
24910,87092849,888,Other,NaN,NaN,"07/06/21 00:00:00,000000000",8.0,0.0,1
...,...,...,...,...,...,...,...,...,...
1125023,87114568,878,Other,NaN,NaN,"07/06/21 00:00:00,000000000",20.0,0.0,1
1129209,87211758,878,Other,NaN,NaN,"07/06/21 00:00:00,000000000",12.0,0.0,1
1130473,87051983,888,Other,NaN,NaN,"07/06/21 00:00:00,000000000",132.0,0.0,1
1130814,87720037,878,Other,NaN,NaN,"07/06/21 00:00:00,000000000",80.0,0.0,1


In [51]:
transactions['TR_SUPPORT_CODE'].value_counts(dropna = False).head(5)

C700    498625
C100    443137
TIS4     80670
C200     26965
T200     22765
Name: TR_SUPPORT_CODE, dtype: int64

In [52]:
transactions['TR_SUPPORT_TYPE'].value_counts(dropna = False)

C      1051899
V        90565
NaN       2394
P         1227
Name: TR_SUPPORT_TYPE, dtype: int64

In [53]:
transactions['TR_SUPPORT_CODE'].loc[(transactions['TR_SUPPORT_TYPE'].isna())].value_counts()
## many are C100, EMET, C700 SUPPORT_CODE

C100    652
EMET    600
C700    556
PL03     62
PL01     37
TIS4     25
R200     18
T200     13
PL31     10
C200      9
WK00      5
H200      4
F200      4
C800      4
TIS7      3
WK01      2
WK10      1
C901      1
X700      1
C500      1
Name: TR_SUPPORT_CODE, dtype: int64

In [54]:
transactions['TR_SUPPORT_TYPE'].loc[(transactions['TR_SUPPORT_CODE']== 'EMET')].sum()
## all EMET have a NaN TR_SUPPORT_TYPE

0

In [55]:
## C100 and C700 are all of TR_SUPPORT_TYPE C
transactions['TR_SUPPORT_TYPE'].loc[(transactions['TR_SUPPORT_CODE']== 'C100') | (transactions['TR_SUPPORT_CODE']== 'C700')].value_counts()

C    940554
Name: TR_SUPPORT_TYPE, dtype: int64

In [56]:
transactions['TR_SUPPORT_TYPE'].loc[transactions['TR_SUPPORT_CODE'].str.contains('C', na=False,regex=False)]

0          C
1          C
3          C
4          C
5          C
          ..
1146079    C
1146081    C
1146082    C
1146083    C
1146084    C
Name: TR_SUPPORT_TYPE, Length: 996528, dtype: object

In [57]:
## 1) select only Fuel transactions
transactions_fuel = transactions[transactions['TR_PRODUCT_TYPE'] == 'Fuel']
## 2) keep only the day (delete the hours/min/sec) and convert to datetime format
transactions_fuel['TR_DATE']=transactions_fuel['TR_DATE'].astype('str').str[0:8]
transactions_fuel['TR_DATE'] = pd.to_datetime(transactions_fuel['TR_DATE'], format='%d/%m/%y')
## 3) sort by EP_CODE, then by date
transactions_fuel = transactions_fuel.sort_values(by=['EP_CODE','TR_DATE'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [58]:
transactions_fuel.dtypes

EP_CODE                     int64
TR_PRODUCT_CODE             int64
TR_PRODUCT_TYPE            object
TR_SUPPORT_CODE            object
TR_SUPPORT_TYPE            object
TR_DATE            datetime64[ns]
TR_AMOUNT                 float64
TR_VOLUME                 float64
TR_COUNT                    int64
dtype: object

In [59]:
transactions_fuel[(transactions_fuel['EP_CODE'] == 87000070) & (transactions['TR_PRODUCT_TYPE'] == 'Fuel')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
51780,87000070,112,Fuel,C700,C,2021-06-05,77.53,59.20,1
404782,87000070,109,Fuel,C700,C,2021-06-08,21.45,54.88,1
786245,87000070,102,Fuel,C700,C,2021-06-08,546.36,433.61,2
367735,87000070,102,Fuel,C700,C,2021-06-10,67.00,52.49,1
825286,87000070,112,Fuel,C700,C,2021-06-10,67.96,51.89,1
627802,87000070,223,Fuel,C700,C,2021-06-11,35.30,26.05,1
934963,87000070,112,Fuel,C700,C,2021-06-12,158.49,121.41,2
1104133,87000070,112,Fuel,C700,C,2021-06-13,131.25,100.65,2
480005,87000070,112,Fuel,C700,C,2021-06-14,40.91,31.59,1
563751,87000070,102,Fuel,C700,C,2021-06-14,504.86,394.24,1


In [60]:
transactions_fuel_na = transactions_fuel[transactions_fuel.isna().any(axis=1)]
transactions_fuel_na['TR_SUPPORT_CODE'].value_counts(dropna=False)
## one SUPPORT_CODE is NaN, the others are EMET which all have a NaN TR_SUPPORT_TYPE
## we should replace the only NaN by , and the NaN in TR_SUPPORT_TYPE by a new value 'E' corresponding to EMET TR_SUPPORT_CODE

EMET    426
NaN       1
Name: TR_SUPPORT_CODE, dtype: int64

In [61]:
transactions_fuel[transactions_fuel['TR_SUPPORT_CODE'].isna()]

,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
521169,87101920,102,Fuel,NaN,C,2021-06-08,84.98,67.58,1


In [62]:
## this customer is not in the customer data table, I just replace the NaN by 'UNK' to not remove the transaction
customer2[customer2['EP_CODE'] == '87101920']

,EP_CODE,EP_STATUS,EP_TYPE,EP_FLEET_TYPE


In [63]:
transactions_fuel['TR_SUPPORT_TYPE'].value_counts(dropna=False)

C      728989
NaN       426
Name: TR_SUPPORT_TYPE, dtype: int64

In [64]:
## 4) replace the NaN in the obs with unknown customerID
transactions_fuel2 = transactions_fuel.fillna(value={'TR_SUPPORT_CODE': 'UNK'})
## 5) delete the TR_PRODUCT_TYPE since it is always 'Fuel' in this table
transactions_fuel2 = transactions_fuel2.drop(columns=['TR_PRODUCT_TYPE'])
## 6) change NaN to E for all EMET obs
transactions_fuel2 = transactions_fuel2.fillna(value={'TR_SUPPORT_TYPE': 'E'})
## 7) change numeric values to numeric types
## pd.to_numeric(transactions_fuel2['TR_AMOUNT'], downcast='float',errors='ignore')

In [65]:
print(transactions_fuel.shape, transactions_fuel2.shape)
## 1 obs and 1 variable deleted

(729415, 9) (729415, 8)


In [66]:
transactions_fuel2.head(5)

,EP_CODE,TR_PRODUCT_CODE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
372404,87000052,102,C700,C,2021-06-09,125.94,100.15,1
51780,87000070,112,C700,C,2021-06-05,77.53,59.20,1
404782,87000070,109,C700,C,2021-06-08,21.45,54.88,1
786245,87000070,102,C700,C,2021-06-08,546.36,433.61,2
367735,87000070,102,C700,C,2021-06-10,67.00,52.49,1


In [67]:
transactions_fuel2.to_csv(path_treated+'table_transactions_fuel.csv', sep =";", index=False, decimal=',' , encoding='UTF-8')

##   Non-Fuel transactions

In [68]:
## 1) select only Other(non-Fuel) transactions
transactions_other = transactions[transactions['TR_PRODUCT_TYPE'] == 'Other']
## 2) convert date as previously
transactions_other['TR_DATE']=transactions_other['TR_DATE'].astype('str').str[0:8]
transactions_other['TR_DATE'] = pd.to_datetime(transactions_other['TR_DATE'], format='%d/%m/%y')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [69]:
transactions_other.head(5)

,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_VOLUME,TR_COUNT
0,87256967,730,Other,C100,C,2021-06-08,11.83,0.0,5
2,87128463,731,Other,TIS4,V,2021-06-08,43.34,0.0,8
4,87243619,730,Other,C100,C,2021-06-08,73.34,0.0,10
5,87681255,730,Other,C100,C,2021-06-09,65.26,0.0,20
7,87631584,731,Other,TIS4,V,2021-06-07,31.75,0.0,3


In [70]:
pd.to_numeric(transactions_other['TR_VOLUME']).sum()
## No Volume for non-fuel transactions => we cand delete this column

0.0

In [71]:
print(transactions_other.shape[0]) ## number of rows
transactions_other.isna().sum()
## SUPPORT_CODE and SUPPORT_TYPE contain many NaN

416670


EP_CODE               0
TR_PRODUCT_CODE       0
TR_PRODUCT_TYPE       0
TR_SUPPORT_CODE    1607
TR_SUPPORT_TYPE    1968
TR_DATE               0
TR_AMOUNT             0
TR_VOLUME             0
TR_COUNT              0
dtype: int64

In [72]:
## This gives the SUPPORT_TYPE when the SUPPORT_CODE contains a C (C100, C700...)
transactions_other['TR_SUPPORT_TYPE'].loc[transactions_other['TR_SUPPORT_CODE'].str.contains('C', na=False,regex=False)].value_counts(dropna=False)
## We can safely assume the NaNs can be turned into C types

C      301056
NaN      1223
Name: TR_SUPPORT_TYPE, dtype: int64

In [73]:
## This gives the SUPPORT_CODE when the SUPPORT_TYPE is P
transactions_other['TR_SUPPORT_CODE'].loc[transactions_other['TR_SUPPORT_TYPE'] == 'P'].value_counts(dropna=False)
## We can replace the NaN by 'PL01'

NaN     1221
PL01       6
Name: TR_SUPPORT_CODE, dtype: int64

In [74]:
## 3) delete TR_PRODUCT_TYPE and TR_VOLUME columns
transactions_other2 = transactions_other.drop(columns=['TR_PRODUCT_TYPE','TR_VOLUME'])
## 4) sort by EP_CODE, then by TR_DATE
transactions_other2 = transactions_other2.sort_values(by=['EP_CODE','TR_DATE'])
## 5) treat the NaN of SUPPORT_TYPE of EMET SUPPORT_CODE into E type, and the NaN left as C type
transactions_other2.loc[(transactions_other2['TR_SUPPORT_TYPE'].isna()) & (transactions_other2['TR_SUPPORT_CODE'] == 'EMET'), 'TR_SUPPORT_TYPE'] = 'E'
transactions_other2.loc[(transactions_other2['TR_SUPPORT_TYPE'].isna()) & (transactions_other2['TR_SUPPORT_CODE'].str.contains('C', na=False,regex=False)), 'TR_SUPPORT_TYPE'] = 'C'
## 6) treat the P SUPPORT_TYPE as PL01 SUPPORT_CODE, and inversely if the CODE contains 'PL', the TYPE is 'P'
transactions_other2.loc[(transactions_other2['TR_SUPPORT_CODE'].isna()) & (transactions_other2['TR_SUPPORT_TYPE'] == 'P'), 'TR_SUPPORT_CODE'] = 'PL01'
transactions_other2.loc[(transactions_other2['TR_SUPPORT_TYPE'].isna()) & (transactions_other2['TR_SUPPORT_CODE'].str.contains('PL', na=False,regex=False)), 'TR_SUPPORT_TYPE'] = 'P'
## 7) I don't want to delete some transactions so, treat the rest with CODE 'UNK' and TYPE 'U' for last NaNs
transactions_other2 = transactions_other2.fillna(value={'TR_SUPPORT_CODE': 'UNK','TR_SUPPORT_TYPE': 'U'})
## 8) transform TR_AMOUNT to float non-comma-separated values
#for index,row in transactions_other2.iterrows():
#   row['TR_AMOUNT']=float(row['TR_AMOUNT'].replace(',','.'))

In [75]:
transactions_other2.head(5)

,EP_CODE,TR_PRODUCT_CODE,TR_SUPPORT_CODE,TR_SUPPORT_TYPE,TR_DATE,TR_AMOUNT,TR_COUNT
973236,87000019,731,TIS4,V,2021-06-05,32.26,6
337978,87000019,730,C100,C,2021-06-06,2.58,1
604554,87000019,731,TIS4,V,2021-06-06,15.01,3
1004029,87000019,741,TIS7,V,2021-06-06,8.33,1
650728,87000019,731,TIS4,V,2021-06-07,15.26,3


In [76]:
#transactions_other2[transactions_other2['TR_SUPPORT_TYPE'].isna()].head(300)

In [77]:
transactions_other2.to_csv(path_treated+'table_transactions_nonfuel.csv', sep =";", index=False, decimal=',' , encoding='UTF-8')

In [78]:
print(f'total transactions : {transactions.shape[0]} \ntotal fuel transactions : {transactions_fuel2.shape[0]} \ntotal non-fuel transactions : {transactions_other2.shape[0]}')

total transactions : 1146085 
total fuel transactions : 729415 
total non-fuel transactions : 416670


## transactions per customer / support / product

In [79]:
## group amounts and volumes by clients/support/product
transactions_fuel_by_client = transactions_fuel[['EP_CODE','TR_AMOUNT','TR_VOLUME']].groupby(['EP_CODE']).sum().round(2)
transactions_fuel_by_support = transactions_fuel[['TR_SUPPORT_TYPE','TR_AMOUNT','TR_VOLUME']].groupby(['TR_SUPPORT_TYPE']).sum().round(2)
transactions_fuel_by_product = transactions_fuel[['TR_PRODUCT_CODE','TR_AMOUNT','TR_VOLUME']].groupby(['TR_PRODUCT_CODE']).sum().round(2)

In [80]:
## export with "," decimal values for power BI
transactions_fuel_by_client.to_csv(path_treated+'table_transactions_fuel_by_client.csv', sep =";", index=True, decimal=',' ,encoding='UTF-8')
transactions_fuel_by_support.to_csv(path_treated+'table_transactions_fuel_by_support.csv', sep =";", index=True, decimal=',' ,encoding='UTF-8')
transactions_fuel_by_product.to_csv(path_treated+'table_transactions_fuel_by_product.csv', sep =";", index=True, decimal=',' ,encoding='UTF-8')

In [81]:
transactions_fuel_by_support.sort_values(['TR_AMOUNT'], ascending = False).head(5)

,TR_AMOUNT,TR_VOLUME
TR_SUPPORT_TYPE,,
C,1.226662e+08,98426517.89


In [82]:
transactions_fuel_by_product.sort_values(['TR_AMOUNT'], ascending = False).head(5)

,TR_AMOUNT,TR_VOLUME
TR_PRODUCT_CODE,,
102,94950296.98,76813430.88
112,16419745.70,12757130.38
223,8536229.67,6466028.41
201,4786001.42,3409393.04
116,3571276.18,2871970.83


# checks for powerBI consistency

In [83]:
transactions_fuel2['TR_AMOUNT'].sum().round(2)

129882665.66

In [84]:
transactions_other2['TR_AMOUNT'].sum().round(2)

13131198.62

In [93]:
transactions_fuel2[['TR_DATE','TR_AMOUNT']].groupby(['TR_DATE']).sum()

,TR_AMOUNT
TR_DATE,
2021-06-05,4501635.48
2021-06-06,2924532.29
2021-06-07,18942482.99
2021-06-08,18684149.12
2021-06-09,18690184.95
2021-06-10,19253542.73
2021-06-11,19615606.59
2021-06-12,4643055.20
2021-06-13,3101110.33


In [95]:
transactions_other2[['TR_DATE','TR_AMOUNT']].groupby(['TR_DATE']).sum()

,TR_AMOUNT
TR_DATE,
2021-06-05,519611.34
2021-06-06,514208.77
2021-06-07,2643347.65
2021-06-08,2280198.68
2021-06-09,2222836.86
2021-06-10,2319673.30
2021-06-11,1605943.62
2021-06-12,343972.70
2021-06-13,211726.46


In [103]:
transactions_fuel2[['TR_PRODUCT_CODE','TR_AMOUNT']].groupby(['TR_PRODUCT_CODE']).sum().sort_values(['TR_AMOUNT'], ascending = False)

,TR_AMOUNT
TR_PRODUCT_CODE,
102,9.495030e+07
112,1.641975e+07
223,8.536230e+06
201,4.786001e+06
116,3.571276e+06
222,7.724286e+05
109,3.591204e+05
115,1.828809e+05
285,1.108950e+05


In [102]:
transactions_other2[['TR_PRODUCT_CODE','TR_AMOUNT']].groupby(['TR_PRODUCT_CODE']).sum().sort_values(['TR_AMOUNT'], ascending = False)

,TR_AMOUNT
TR_PRODUCT_CODE,
730,6.679400e+06
731,2.548067e+06
739,1.172254e+06
303,5.372315e+05
563,3.503025e+05
...,...
9012,4.200000e-01
881,-5.636000e+01
722,-6.000000e+01


In [85]:
customer2['EP_FLEET_TYPE'].value_counts()

VL_PUR      96536
VL_MIXTE    39836
PL_MIXTE     3605
PL           2672
PL_VIP       1850
Name: EP_FLEET_TYPE, dtype: int64

In [86]:
customer2['EP_TYPE'].value_counts()

Standard                      133115
Administration                 10225
International organisation       678
Third party intermediary         472
Name: EP_TYPE, dtype: int64

In [88]:
support2['VEHICLE_TYPE'].value_counts()

04-OTHER                                  98046
03-PASSENGER VEHICLE                      87468
02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE    82669
01-HEAVY GOODS VEHICLE                    20786
Name: VEHICLE_TYPE, dtype: int64

In [87]:
support2['SUPPORT_TYPE'].value_counts()

C    228809
V     51061
P      9099
Name: SUPPORT_TYPE, dtype: int64